In [17]:
import cv2
import numpy as np
from numpy.core.arrayprint import printoptions
import skimage.io as io
from time import sleep

In [18]:
cap = cv2.VideoCapture('trimmed1.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
imgTest=[]
picList=[]
# Read until video is completed
i=200
while(cap.read()):
  # Capture frame-by-frame
  i-=1
  if(i==0):
    break

  ret, frame = cap.read()
  if ret == True:
    if(i==150):
      imgTest.append(frame)
    if(i==2):
      print(frame.shape)
    picList.append(frame)
    # Display the resulting frame
    #cv2.imshow('Frame',frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
# cap.release()

# Closes all the frames
# cv2.destroyAllWindows()


(858, 984, 3)


In [19]:
arr = np.asarray(picList)

print(arr.shape)

meanimage=np.sum(arr,axis=0)//len(picList)
print(meanimage.shape)
image = cv2.cvtColor(np.float32(meanimage), cv2.COLOR_BGR2RGB)

#io.imshow(image)
cv2.imshow("lol",np.uint8(meanimage))
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imshow( meanimage)
  
# waits for user to press any key
# (this is necessary to avoid Python kernel form crashing)



(199, 858, 984, 3)
(858, 984, 3)


In [23]:
img1=cv2.cvtColor(np.uint8(meanimage), cv2.COLOR_BGR2GRAY)
img2=cv2.cvtColor(np.uint8(imgTest[0]), cv2.COLOR_BGR2GRAY)
#imgdiff=cv2.absdiff(np.uint8(meanimage), np.uint8(imgTest[0]))
imgdiff=cv2.absdiff(np.uint8(img1), np.uint8(img2))

#imgdiff = np.asarray(imgdiff, np.float64)
_, thresh = cv2.threshold(imgdiff, 50, 255, cv2.THRESH_BINARY)
kernel=np.ones((10,10),np.uint8)
thresh = cv2.dilate(thresh,kernel,1)

#io.imshow(thresh)
#pp=cv2.equalizeHist(imgdiff)
cv2.imshow( "lol2", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# load a video
video = cv2.VideoCapture('trimmed1.mp4')

# You can set custom kernel size if you want.
kernel = None

# Initialize the background object.
#backgroundObject = cv2.createBackgroundSubtractorMOG2(history=256,detectShadows = True)
first_time=True
while True:
    
    # Read a new frame.
    ret, frame = video.read()

    # Check if frame is not read correctly.
    if not ret:
        break

    # Apply the background object on the frame to get the segmented mask. 
    #fgmask = backgroundObject.apply(6)
    img2=cv2.cvtColor(np.uint8(frame), cv2.COLOR_BGR2GRAY)
    imgdiff=cv2.absdiff(np.uint8(img1), np.uint8(img2))

    #imgdiff = np.asarray(imgdiff, np.float64)
    _, thresh = cv2.threshold(imgdiff, 50, 255, cv2.THRESH_BINARY)
    fgmask=thresh
    temp=np.copy(fgmask)
    #_, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    


    kernel=np.ones((6,18),np.uint8)



    fgmask = cv2.erode(fgmask,kernel,1)
    fgmask = cv2.erode(fgmask,kernel,1)
    fgmask = cv2.dilate(fgmask,kernel,1)
    fgmask = cv2.dilate(fgmask,kernel,1)
    fgmask = cv2.dilate(fgmask,kernel,1)
    fgmask = cv2.dilate(fgmask,kernel,1)
    fgmask = cv2.dilate(fgmask,kernel,1)
    #fgmask = cv2.dilate(fgmask,kernel,1)
    #fgmask = cv2.dilate(fgmask,kernel,1)
    #fgmask = cv2.erode(fgmask,kernel,1)
    
    kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(10,10))
    fgmask = cv2.dilate(fgmask,kernel,1)
    #fgmask = cv2.erode(fgmask,kernel,1)
    #fgmask = cv2.erode(fgmask,kernel,1)
    #fgmask = cv2.dilate(fgmask,kernel,1)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    # Create a copy of the frame to draw bounding boxes around the detected cars.
    frameCopy = frame.copy()
    
    # loop over each contour found in the frame.
    for cnt in contours:
        
        # Make sure the contour area is somewhat higher than some threshold to make sure its a car and not some noise.
        if cv2.contourArea(cnt) >100 :
            
            # Retrieve the bounding box coordinates from the contour.
            x, y, width, height = cv2.boundingRect(cnt)
            
            # Draw a bounding box around the car.
            cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)
            
            # Write Car Detected near the bounding box drawn.
            #cv2.putText(frameCopy, 'Car Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
    
    # Extract the foreground from the frame using the segmented mask.
    foregroundPart = cv2.bitwise_and(frame, frame, mask=fgmask)
        
    # Stack the original frame, extracted foreground, and annotated frame. 
    stacked = np.hstack((frame, foregroundPart, frameCopy))

    # Display the stacked image with an appropriate title.
    cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked, None, fx=0.5, fy=0.5))
    #cv2.imshow('initial Mask', initialMask)
    #cv2.imshow('Noisy Mask', noisymask)
    #cv2.imshow('Clean Mask', fgmask)

    sleep(0.05)
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xff
    
    # Check if 'q' key is pressed.
    if k == ord('q'):
        #plt_image(fgmask)
        # Break the loop.
        break

# Release the VideoCapture Object.
video.release()

# Close the windows.q
cv2.destroyAllWindows()